# The 12 Most Awesome Growing Job Markets in the U.S.

<p>Based on the U.S. Bureau of Labor Statistic’s [latest QCEW data]("https://www.bls.gov/cew/") released Sept. 6.</p>

In [1]:
import pandas as pd

<p>We start by importing the dataset.</p>

In [42]:
df = pd.read_csv("/Users/alexmahadevan/Desktop/Projects/Wages/2017.q1-q1.by_industry/2017.q1-q1 10 10 Total, all industries.csv")

<p>Now we’ll examine the variables we want to isolate in the analysis.</p>

In [43]:
df.head(1)

,area_fips,own_code,industry_code,agglvl_code,size_code,year,qtr,disclosure_code,area_title,own_title,...,oty_month3_emplvl_chg,oty_month3_emplvl_pct_chg,oty_total_qtrly_wages_chg,oty_total_qtrly_wages_pct_chg,oty_taxable_qtrly_wages_chg,oty_taxable_qtrly_wages_pct_chg,oty_qtrly_contributions_chg,oty_qtrly_contributions_pct_chg,oty_avg_wkly_wage_chg,oty_avg_wkly_wage_pct_chg
0,28055,1,10,71,0,2017,1,NaN,"Issaquena County, Mississippi",Federal Government,...,-1,-100.0,-659.0,-6.4,0,0.0,0,0.0,1439,180.8


<p>We’re only looking at private employment in this analysis. So we’ll go ahead and isolate that sector here.</p>

In [44]:
df = df[df.own_code == 5]

<p>Now let’s throw out some of the smaller counties</p>

In [45]:
df_large = df[df.qtrly_estabs_count >= 3592 ]

<p>Now, we’ll isolateonly the counties, since they included MSAs in the dataset.</p>

In [46]:
df_county = df_large[df_large.agglvl_code==71]

In [47]:
pd.options.mode.chained_assignment = None

<p>Now, we’re looking at the percentage change in average weekly wage over 2016, the percent change in new establishments (businesses) and the increase — or decrease — in employment.</p>

<p>To do that, we apply the Z-score method to those variables to normalize them. That way we can compare apples to apples.</p>

In [48]:
pd.options.mode.chained_assignment = None

In [49]:
df_county['zWage'] = (df_county.oty_avg_wkly_wage_pct_chg - df_county.oty_avg_wkly_wage_pct_chg.mean())/ df_county.oty_avg_wkly_wage_pct_chg.std()

In [53]:
df_county['zNewBusiness'] = (df_county.oty_qtrly_estabs_count_pct_chg - df_county.oty_qtrly_estabs_count_pct_chg.mean())/ df_county.oty_qtrly_estabs_count_pct_chg.std()

<p>There’s one problem: we don’rt have a variable that takes into account employment change throughout the quarter. Let’s just take the year-over-year change in employment for each month and find the average.</p>

In [50]:
df_county['avgeemployment'] = (df_county.oty_month1_emplvl_pct_chg + df_county.oty_month2_emplvl_pct_chg+ df_county.oty_month3_emplvl_pct_chg)/3

<p>Back to Z-scores.</p>

In [51]:
df_county['zEmployment'] = (df_county.avgeemployment - df_county.avgeemployment.mean())/ df_county.avgeemployment.std()

<p>Now we’ll average the Z-scores to find the overall rank.</p>

In [54]:
df_county['zRank'] = (df_county.zWage + df_county.zNewBusiness + df_county.zEmployment) / 3

<p>Now we rank them.</p>

In [23]:
county_business_rank = df_county.sort_values('zRank' , ascending=False)

<p>Now we have our additional dataset.</p>

In [24]:
county_business_rank.to_csv("/Users/alexmahadevan/Desktop/Projects/Wages/analysis.csv")

<p>We cleaned that up a little in Excel, now let’s load it back in with another dataset containing cost-of-living data.</p>

In [25]:
county_data = pd.read_csv("/Users/alexmahadevan/Desktop/Projects/Wages/county data.csv")

In [26]:
wage_data = pd.read_csv("/Users/alexmahadevan/Desktop/Projects/Wages/with cities.csv")

<p>Now we merge the two datasets.</p>

In [28]:
merged = pd.merge(wage_data, county_data, on="CTY")

<p>And finally, we output our merged dataset (still sorted with the biggest booming job markets.</p>

In [29]:
merged.to_csv("/Users/alexmahadevan/Desktop/wage.csv")

<p>We finished by looking at the median rent cost as the affordability factor, and adding it to the ranking in Excel. We threw out some of the top cities (Bye, bye expensive Franklin, Tennessee!).</p>
<p>Thank you for reading, and make sure to check out [The Penny Hoarder]("https://www.thepennyhoarder.com") for tons of content to put more money in your pocket!</p>